In [1]:
import os
import re
import pickle
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
tempfile = pd.read_csv("sina.csv")

In [4]:
cookieFile = "cookie.pkl"
cookies = None

if os.path.exists(cookieFile):
    with open(cookieFile, "rb") as inFile: cookies = pickle.load(inFile)

In [5]:
driver = webdriver.Firefox()
if cookies is not None:
    driver.get("https://weibo.com")
    for co in cookies: driver.add_cookie(co)
driver.get("https://weibo.com")
# _ = input("press ENTER after login to weibo")

In [6]:
def wait_for_element(driver, className, timeout=30):
    try:
        el = EC.presence_of_element_located((By.CLASS_NAME, className))
        WebDriverWait(driver, timeout).until(el)
    except TimeoutException:
        print("Timeout")
        return False
    finally:
        return True

In [7]:
def clean_weibo_content(content):
    #content = unicodedata.normalize("NFKD", content)
    #content = content.replace('\u200b', ' ').strip()
    content = re.sub(r"[^\u4e00-\u9fff0-9a-zA-Z@_]", " ", content)
    content = " ".join(content.split())
    return content

In [12]:
def collect_weibo(driver,tempfile):
    all_data = []
    for a,b in zip(tempfile["名字"], tempfile["url"]):
        driver.get(b+"?is_hot=1")
        if not wait_for_element(driver, "WB_frame_c"): continue
        if not wait_for_element(driver, "WB_detail"): continue
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        time = [i.get("title") for i in soup.select("div.WB_frame_c div.WB_detail a.S_txt2")[::2]][:10]
        content = [clean_weibo_content(i.getText()) for i in soup.select("div.WB_frame_c div.WB_detail div.WB_text")][:10]
        rewebo = [int(i.getText()) if i.getText().isnumeric() else 0 for i in soup.select("div.WB_frame_c div.WB_feed_handle span.line span em")[3::8]][:10]
        comment = [int(i.getText()) if i.getText().isnumeric() else 0 for i in soup.select("div.WB_frame_c div.WB_feed_handle span.line span em")[5::8]][:10]
        thumb = [int(i.getText()) if i.getText().isnumeric() else 0 for i in soup.select("div.WB_frame_c div.WB_feed_handle span.line span em")[7::8]][:10]
        row = [a,b]
        for i in zip(content,time,rewebo,comment,thumb):
            row = row + list(i)
        all_data.append(row)
    label = ["名字","url"]
    for x in range (10):
        label+=["内容"+str(x),"时间"+str(x),"转发"+str(x),"评论"+str(x),"点赞"+str(x)]
    all_data = pd.DataFrame(all_data,columns = label)
    return all_data

In [13]:
data = collect_weibo(driver,tempfile)

Timeout
Timeout


In [15]:
data.to_csv("sinaContent.csv", index=False, encoding='utf-8-sig')

In [16]:
driver.get("https://weibo.com")
with open(cookieFile, "wb") as outFile:
    pickle.dump(driver.get_cookies(), outFile)

driver.close()